# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Описание данных\
**Признаки**

    DateCrawled — дата скачивания анкеты из базы
    VehicleType — тип автомобильного кузова
    RegistrationYear — год регистрации автомобиля
    Gearbox — тип коробки передач
    Power — мощность (л. с.)
    Model — модель автомобиля
    Kilometer — пробег (км)
    RegistrationMonth — месяц регистрации автомобиля
    FuelType — тип топлива
    Brand — марка автомобиля
    NotRepaired — была машина в ремонте или нет
    DateCreated — дата создания анкеты
    NumberOfPictures — количество фотографий автомобиля
    PostalCode — почтовый индекс владельца анкеты (пользователя)
    LastSeen — дата последней активности пользователя

**Целевой признак**

    Price — цена (евро)

## Подготовка данных

Загрузим необходимые библиотеки

In [ ]:
import re
import os

from IPython.display import display

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#plt.style.use('dark_background') # Темная тема для matplotlib

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.utils import shuffle # Миксер
from sklearn.utils.validation import check_is_fitted


# !pip install skimpy
# from skimpy import clean_columns

!pip install plotly==5.5.0 # Для тренажера так как ошибка textauto в heatmap

!pip install lightgbm
from lightgbm import LGBMRegressor as lgbmr

!pip install catboost
from catboost import CatBoostRegressor

Загрузим данные

In [ ]:
pth1 = '/content/drive/MyDrive/data/autos.csv' # Для Colab
pth2 = '/datasets/autos.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
    print("Датасет успешно загружен c GoogleDisc")
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
    print("Датасет успешно загружен c Яндекс.Практикум")
else:
    print("Проверьте правильность пути к датасету")

In [ ]:
df.head()

Переименуем столбцы

In [ ]:
new_columns_name=[]
for column in df.columns:
    new_columns_name.append(re.sub(r'(?<!^)(?=[A-Z])', '_', column).lower())
df.columns = new_columns_name

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

Проверим на пропуски

In [ ]:
display(pd.DataFrame(round((df.isna().mean()*100),2), columns=['NaNs, %']).style.format(
    '{:.2f}').background_gradient('coolwarm'))

Довольно значительное количество пропусков во всех столбцах где они были обнаружены, поэтому просто удалить не получится попробуем обработать их далее

Проверим на дубликаты

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

Проверим столбцы `last_seen`, `date_created`, `date_crawled`

In [ ]:
df['date_created'] = pd.to_datetime(df['date_created'])
df['last_seen'] = pd.to_datetime(df['last_seen'])
df['date_crawled'] = pd.to_datetime(df['date_crawled'])

In [ ]:
df[['last_seen', 'date_created', 'date_crawled']].apply([min, max], axis=0)

In [ ]:
(df['date_crawled'] - df['last_seen']).mean()

Дата скачивания анкеты была проставлена раньше чем дата последнего визита пользователя
Похоже что эти столбцы не принесут польезной информации как и столбец с датой создания анкеты `date_created`, удалим их, заодно избавимся от столбца с нулевыми значениями, `number_of_pictures`

In [ ]:
df = df.drop(['last_seen', 
              'date_created', 
              #'date_crawled', оставим для фильтра года месяца регистрации
              'number_of_pictures'], axis=1)

In [ ]:
df.describe()

В цене есть "0", оправданием может служить плохое техническое состояние или необходимость уплаты налога при утилизации. Не исключаем также вероятность автоматического заполнения пропусков "0", как и для столбца `power`, а также месяца регистрации `registration_month`

Проверим какие марки встречаются у нас, с количеством моделей

In [ ]:
df.groupby('brand')['model'].nunique()

sonstige_autos - автомобили бывшие в эксплуатации, скорее всего используется вместо неизвестных значений, также заменим rover = land_rover

In [ ]:
df.loc[df['brand'] == 'land_rover', 'brand'] = 'rover'

Проверим есть ли совпадающие названия у разных производителей, если есть добавим к ней название производителя

In [ ]:
for model in df['model'].unique():
    if df[df['model'] == model]['brand'].nunique() > 1:
        print( model, df[df['model'] == model]['brand'].unique())
        df['model'].where(~(df['model'] == model), other=df.brand + ' ' + model, inplace=True)

Заменим пропуски в названиях моделей на название бренда добавив unknown_model

In [ ]:
df['model'].where(~(df['model'].isna()), other=df['brand'] +' unknown_model', inplace=True)

В названиях моделей встречается reihe - с немецкого переводится как "линия", значит можно предположить что датасет немецкий.
Система присвоения индексов совпадает с принятой почтой Германии
Заменим индексы на названия регионов

In [ ]:
df['zone'] = df['postal_code'] // 10000
df = df.drop('postal_code', axis=1)

index_dict = {0: "Dresden_Leipzig", 
              1: "Berlin", 
              2: "Hamburg_Bremen", 
              3: "Hannover", 
              4: "Duesseldorf_Dortmund", 
              5: "Koeln", 
              6: "Frankfurt", 
              7: "Stuttgart", 
              8: "Muenchen", 
              9: "Nuernberg_Bayer"}

df.zone = df.zone.replace(index_dict)

In [ ]:
zone_model_pivot = df.pivot_table(index='zone', values='model', aggfunc='count')

fig = px.bar(zone_model_pivot, y="model")
fig.update_layout(bargap=0.05, 
                  title_text='Количество объявлений по регионам',
                  xaxis_title_text='Регион',
                  yaxis_title_text='Количество')
fig.show()

Посмотрим на пробег автомобилей в датасете

In [ ]:
df.kilometer.unique()

Похоже, что пробег пользователь выбирает из представленных доступных значений

In [ ]:
fig = px.histogram(df, x="kilometer")
fig.update_layout(bargap=0.05, 
                  title_text='Пробег автомобилей',
                  xaxis_title_text='Год',
                  yaxis_title_text='Количество')
fig.show()

Значительная часть автомобилей представлена с пробегом 150 000км. И так как это максимальное доступное число скорее всего в реальности пробег у автомобилей значительно выше. 

Проверим столбец с годом регистрации `registration_year`

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=df['registration_year'], name=' ',
                marker_color = 'indianred'))
fig.update_layout(title_text='Годы регистраций автомобилей')
fig.update_traces(orientation='h')
fig.update_xaxes(range=[1960, 2020])
fig.show()

In [ ]:
df.query('1950 > registration_year')

In [ ]:
df.query('2017 < registration_year')

Данные старше 1960 похожи на выбросы удалим их, а также объявления с годом регистрации старше 2016 года(по дате скачивания анкеты)

In [ ]:
df = df.query('1960 < registration_year < 2017')

In [ ]:
df = df[~((df['registration_year'] == 2016) & (df['registration_month'] > 4))]

In [ ]:
df.info()

In [ ]:
fig = px.histogram(df, x="registration_year")
fig.update_layout(bargap=0.05, 
                  title_text='Количество объявлений по годам регистрации автомобиля',
                  xaxis_title_text='Год',
                  yaxis_title_text='Количество')
fig.show()

Проверим столбец с месяцем регистрации `registration_month`

In [ ]:
fig = px.histogram(df, x="registration_month")
fig.update_layout(bargap=0.05, 
                  title_text='Количество объявлений по месяцам',
                  xaxis_title_text='Месяц',
                  yaxis_title_text='Количество')
fig.show()

In [ ]:
display(df.query('registration_year > 1980').pivot_table(index='registration_month', 
                                                 columns='registration_year', 
                                                 values='model',  
                                                 aggfunc='count').style.format(
    '{:.2f}').background_gradient('coolwarm'))

Нулем в месяцах предположительно, заполняются пропущенные значения. Значительное количество нулевых значений встречается до 2000 года и после 2015. Данные о продажах по месяцам могут показать некоторую сезонность в коллебаниях цен, но учитывая число пропусков скорее всего эта информация будет не доступна. Удалим столбец.

In [ ]:
df = df.drop(['registration_month'], axis=1)

Посмотрим распределение цен на автомобили

In [ ]:
df.price.describe()

In [ ]:
fig = px.histogram(df, x="price")
fig.update_layout(bargap=0.05, 
                  title_text='Количество объявлений по цене',
                  xaxis_title_text='Цена',
                  yaxis_title_text='Количество')
fig.show()

Видно большое количество значений около 0. Либо таким образом заполнены пропуски когда владелец не указывает цену в объявлении либо автомобили в таком состоянии. Что владелец с радостью избавиться от авто(возможно налог на утилизацию).

Отбросим нулевые значения так как они отрийательно влияют на качество обучения модели

In [ ]:
df = df.query('price > 0')

Проверим распределение мощности двигателей

In [ ]:
df.power.describe()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=df['power'], name=' ',
                marker_color = 'indianred'))
fig.update_layout(title_text='Распределение мощности двигателей')
fig.update_traces(orientation='h')
fig.update_xaxes(range=[-50, 800])

fig.show()

Опять значительное число нулевых значений, примем за выбросы все что выше 460 л.с

In [ ]:
df = df.query('power < 460')

Проверим столбец с типом транспортного средства

In [ ]:
df.vehicle_type.unique()

Есть значение other, вероятно для более редких типов кузова. Заменим пропуски на unknown

In [ ]:
df.vehicle_type = df.vehicle_type.fillna('unknown')

In [ ]:
df.head()

Проверим встречающиеся типы коробок передач

In [ ]:
df.gearbox.unique()

Заполним пропуски

In [ ]:
df.gearbox = df.gearbox.fillna('non_spec')

In [ ]:
df.fuel_type.unique()

In [ ]:
fig = px.histogram(df, x="fuel_type")
fig.update_layout(bargap=0.05, 
                  title_text='Тип топлива',
                  xaxis_title_text='Топливо',
                  yaxis_title_text='Количество')
fig.show()

Petrol и gasoline слова синонимы для бензина, который преобладает в датасете. Явно не хватает дизельного топлива возможно оно попадает в категорию other.
Пропуски заполним на неизвестный тип топлива

In [ ]:
df['fuel_type'].where(~(df['fuel_type'] == 'gasoline'), other='petrol', inplace=True)
df.fuel_type = df.fuel_type.fillna('unkn_fuel')

Перейдем к столбцу `repaired`

In [ ]:
df.repaired.unique()

Заменим пропуски и посмотрим на распределение

In [ ]:
df.repaired = df.repaired.fillna('unk_condition')

In [ ]:
fig = px.histogram(df, x="repaired")
fig.update_layout(bargap=0.05, 
                  title_text='Состояние автомобиля',
                  xaxis_title_text='Состояние',
                  yaxis_title_text='Количество')
fig.show()

В основном в выборке представлены в состоянии No, тоесть над ней не проводился серьёзный ремонт. Будем надеяться, что эта колонка также отражает и текущее состояние автомобиля.

Добавим новый признак показывающий насколько заявленный пробег для данного автоотличается от среднего пробега для тойже марки того же года

In [ ]:
df['med_model_year_km'] = np.nan
df['med_model_year_km'] = \
(df['med_model_year_km'].fillna(df.groupby(['model', 'registration_year'])['kilometer']\
                            .transform('median')) - df['kilometer']).astype(int) #mean хуже

In [ ]:
fig = px.histogram(df, x='med_model_year_km', nbins=200)
fig.update_layout(bargap=0.05, 
                  title_text='Разница пробега автомобиля и медианным пробегом для данной модели и года',
                  xaxis_title_text='Разница пробега',
                  yaxis_title_text='Количество')
fig.show()

Проверим на мультиколлениарность признаков

In [ ]:
corr_matrix = df.corr().round(2)
    
fig = px.imshow(corr_matrix,
                text_auto=True,
                width=600, 
                height=600,
                color_continuous_scale='Hot')
    
fig.update_xaxes(side="top")
fig.update_layout(title="Корреляционная матрица параметров")
fig.show()

Средняя зависимость только между новым признаком и пробегом

In [ ]:
fig = px.scatter_3d(df.sample(n=3000), 
                    x='registration_year', 
                    y='kilometer', 
                    z='power', 
                    color='price',
                    opacity=0.7)
fig.update_traces(marker_size = 3)

fig.show()

В целом, чем новее и мощнее тем выше цена. Встречаются редкие экземпляры до 1980 по относительно высокой цене

In [ ]:
pd.DataFrame({'NaNs, %': round((df.isna().mean()*100),2), 'NaNs, qty': df.isna().sum()}).style.format('{:.2f}').background_gradient('coolwarm')

Все пропуски заполнены

**Вывод**
В данном разделе мы провели ознакомление и обработку данных для обучения модели.

В исходном датасете нам было представлено 354369 объектов с 16 признаками.

`date_crawled`, `date_created`, `last_seen` - даты создания, скачивания, последнего посещения по сути бесполезны для предсказания цены автомобиля, более полезным было бы время подачи и время снятия объявления - по ним косвенно можно предположить адекватность цены

`price` - цена распределена от 0 до 20000, много значений сконцентрировано около 0, возможно либо так заполнились пропуски либо как вариант высокий налог на утилизацию и люди даром отдают машину, еще возможно если продажа идет в формате некого аукциона возможно не поступило ни одного предложения

`registration_year` - год регистрации автомобиля, были удалены выбросы. В основном пик продаж приходится на 2003 год.

`power` - мощность автомобиля, удалены выбросы свыше 450 лс (для серийных автомобилей примем, что большие значения редкость). Много значений около 0. Возможно заполнение пропусков или при состояние авто не позволяет установить истинную мощность

`brand` - представлены основные бренды существующие на рынке. Удалены совпадающие имена. Также обнаружена категория `sonstige_autos` - преположительно являющаяся заменой пропусков, или неизвестных значений(для нее есть одна разновидность модели other)

`model` - обработаны повторы названий одинаковых моделей разных производителей, заполнены пропуски

`kilometer` - по столбцу с пробегом установлено, что большинство автомобилей имеют пробег 150000 км, при учете что пользователь выбирает из доступныхх значений и 150000км максимум, следовательно автомобили имеют пробег выше.

`registration_month` - столбец удален, можно было бы получить сезонные колебания цен, но учитывая пропуски, эта информация была бы утеряна

`postal_code` - было определено что датасет немецкого происхождения и значения индекса были заменены на названия регионов

`repaired` - на большинстве автомобилей не проводилось серьезного ремонта. Не однозначная категория "NotRepaired — была машина в ремонте или нет" предположим, что подразумевается серьезный ремонт(восстановление)

`fuel_type`- объеденены синонимы 'petrol' и 'gasoline', заполнены пропуски, отсутствует категория дизельных авто, вероятно отнсятся к other или к пропускам

`gearbox` - заменены пропуски

`vehicle_type` - заменены пропуски

`number_of_pictures` - признак с 0, удалили

Данные подготовлены переходим к обучению моделей

## Обучение моделей

Подготовим данные для обучения моделей

In [ ]:
X, y = df.drop(['price'], axis=1), df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    random_state=47,
    shuffle=True)

print('Обучающая выборка:')
print('Доля от общего размера:', round(y_train.shape[0]/df.shape[0], 2))
print('Cтрок, столбцов:', X_train.shape)
print('Доля целевого признака:', round(y_train.mean(), 3))
print('- '*10)


print('Тестовая выборка:')
print('Доля от общего размера:', round(y_test.shape[0]/df.shape[0], 2))
print('Cтрок, столбцов:', X_test.shape)
print('Доля целевого признака:', round(y_test.mean(), 3))
print('-'*30)

Разобьем данные на категориальные и числовые обработка для них будет идти по разному

In [ ]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Категориальные признаки: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Числовые признаки: {numerical}")

Подготовим pipeline. Две модели на градиентном бустинге одну на основе линейной регрессии

In [ ]:
cat_pipe = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

num_pipe = Pipeline([
    ('scaler', RobustScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipe, categorical),
    ('num', num_pipe, numerical)
])

pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ("regressor", lgbmr())
    ]
)

param_grid = [
    {
        'regressor': [CatBoostRegressor(random_state=47, 
                                        eval_metric='RMSE', 
                                        silent=True, 
                                        early_stopping_rounds=4)],
        'regressor__depth': [6, 8],
        'regressor__learning_rate': [0.25, 0.4],
        'regressor__iterations': [100, 200]
    },
    {
        'regressor': [lgbmr(random_state=47, metric='rmse')],
        'regressor__learning_rate': [0.4, 0.6],
        'regressor__n_estimators': [100, 150],
    },
    {
        'regressor': [Ridge(random_state=47)],
        'regressor__alpha': [ 1, 3, 5],
        'regressor__max_iter': [1000, 5000],
        'regressor__tol': [0.0001, 0.00001]
    }

]

grid = RandomizedSearchCV(pipe, param_grid, n_iter=30 ,cv=3, scoring = 'neg_mean_squared_error', verbose=3)

Обучим модели и замерим время обучения, замер скорости обучения будем осуществлять с помощью команды %%time, в дальнейшем из таблицы с результатами выберем время обучения лучших моделей

In [ ]:
%%time
grid.fit(X_train, y_train)

## Анализ моделей

Посмотри на результаты работы моделей по среднеквадратичному значению ошибки

In [ ]:
result = pd.DataFrame(grid.cv_results_)
result['mean_test_score'] = (result['mean_test_score']*-1)**0.5

result = result[['param_regressor',
                 'mean_fit_time', 
                 'mean_score_time', 
                 'mean_test_score']].sort_values('mean_test_score').head()
result

Примерно одинаковый результат у LGBMRegressor и CatBoostRegressor, но учитывая ожидаемое среднее время получения результата, CatBoostRegressor значительно выигрывает

Теперь отсортируем таблицу результатов по ожидаемому среднему времени предсказания

In [ ]:
result = pd.DataFrame(grid.cv_results_)
result['mean_test_score'] = (result['mean_test_score']*-1)**0.5

result[['param_regressor',
        'mean_fit_time', 
        'mean_score_time', 
        'mean_test_score']].sort_values('mean_score_time').head()

Здесь безусловно выигрывает Ridge, но со значительно большим среднеквадратичным отклонением

Для оценки важности признаков модели воспользуемся следующим решением [GitHub](https://github.com/scikit-learn/scikit-learn/issues/12525 "GitHub")  (чтобы вытащить названия признаков из OHE) 

In [ ]:
def get_column_names_from_ColumnTransformer(column_transformer, clean_column_names=False, verbose=True):  

    """
    Reference: Kyle Gilde: https://github.com/kylegilde/Kaggle-Notebooks/blob/master/Extracting-and-Plotting-Scikit-Feature-Names-and-Importances/feature_importance.py
    Description: Get the column names from the a ColumnTransformer containing transformers & pipelines
    Parameters
    ----------
    verbose: Bool indicating whether to print summaries. Default set to True.
    Returns
    -------
    a list of the correct feature names
    Note:
    If the ColumnTransformer contains Pipelines and if one of the transformers in the Pipeline is adding completely new columns,
    it must come last in the pipeline. For example, OneHotEncoder, MissingIndicator & SimpleImputer(add_indicator=True) add columns
    to the dataset that didn't exist before, so there should come last in the Pipeline.
    Inspiration: https://github.com/scikit-learn/scikit-learn/issues/12525
    """

    assert isinstance(column_transformer, ColumnTransformer), "Input isn't a ColumnTransformer"
    
    check_is_fitted(column_transformer)

    new_feature_names, transformer_list = [], []

    for i, transformer_item in enumerate(column_transformer.transformers_): 
        transformer_name, transformer, orig_feature_names = transformer_item
        orig_feature_names = list(orig_feature_names)

        if len(orig_feature_names) == 0:
            continue

        if verbose: 
            print(f"\n\n{i}.Transformer/Pipeline: {transformer_name} {transformer.__class__.__name__}\n")
            print(f"\tn_orig_feature_names:{len(orig_feature_names)}")

        if transformer == 'drop':
            continue

        if isinstance(transformer, Pipeline):
            # if pipeline, get the last transformer in the Pipeline
            transformer = transformer.steps[-1][1]

        if hasattr(transformer, 'get_feature_names_out'):
            if 'input_features' in transformer.get_feature_names_out.__code__.co_varnames:
                names = list(transformer.get_feature_names_out(orig_feature_names))
            else:
                names = list(transformer.get_feature_names_out())
        elif hasattr(transformer, 'get_feature_names'):
            if 'input_features' in transformer.get_feature_names.__code__.co_varnames:
                names = list(transformer.get_feature_names(orig_feature_names))
            else:
                names = list(transformer.get_feature_names())

        elif hasattr(transformer,'indicator_') and transformer.add_indicator:
            # is this transformer one of the imputers & did it call the MissingIndicator?

            missing_indicator_indices = transformer.indicator_.features_
            missing_indicators = [orig_feature_names[idx] + '_missing_flag'\
                                  for idx in missing_indicator_indices]
            names = orig_feature_names + missing_indicators

        elif hasattr(transformer,'features_'):
            # is this a MissingIndicator class? 
            missing_indicator_indices = transformer.features_
            missing_indicators = [orig_feature_names[idx] + '_missing_flag'\
                                  for idx in missing_indicator_indices]

        else:

            names = orig_feature_names

        if verbose: 
            print(f"\tn_new_features:{len(names)}")
            print(f"\tnew_features: {names}\n")

        new_feature_names.extend(names)
        transformer_list.extend([transformer_name] * len(names))

    transformer_list, column_transformer_features = transformer_list, new_feature_names

    if clean_column_names:
        new_feature_names = list(clean_columns(pd.DataFrame(columns=new_feature_names)).columns)
    
    return new_feature_names

In [ ]:
dset = pd.DataFrame()
dset['attr'] = get_column_names_from_ColumnTransformer(grid.best_estimator_.named_steps['preprocessor'], verbose=False)
dset['importance'] = grid.best_estimator_.named_steps["regressor"].feature_importances_

dset = dset.sort_values(by='importance').tail(30)

plt.figure(figsize=(16, 14))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature Importances', fontsize=20, fontweight='bold', pad=20)
plt.xlabel('Importance', fontsize=14, labelpad=20)
plt.show()

In [ ]:
print(f"Наиболее важными параметрами влияющими на цену автомобиля по расчетам :\n \
{dset.sort_values(by='importance')['attr'].tail(3)}")

Проверим работу модели на тестовой выборке

In [ ]:
%%time
y_final = grid.best_estimator_.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_final)**0.5

На тестовой выборке модель показывает результат сопоставимый с результатами при кроссвалидации

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)
y_pred = dummy_regr.predict(X_test)
mean_squared_error(y_test, y_pred)**0.5

В сравнении с Dummy моделью также показывает эффективность 

На основании совокупности показанных результатов скорости предсказания и среднеквадратичного отклонения предсказанных результатов рекомендуется выбрать CatBoostRegressor с ниже указанными параметрами

In [ ]:
grid.best_estimator_.named_steps["regressor"].get_all_params()

**Вывод**

В данной работе нами предложена модель предсказания цены автомобиля для приложения сервиса по продаже автомобилей с пробегом «Не бит, не крашен».

На этапе обработки данных избавились от столбцов с датой скачивания анкеты, даты последней активности пользователя и даты регистрации как мало информативные. А также столбец с месяцем регистрации авто, так как из-за обилия нулевых значений адекватно заменить не получилось бы. \
Нами были удалены явные дубликаты, а также дубликаты в марках автомобилей, типе топлива.
Для каждой марки при совпадении названии модели создана своя замена.
В столбце с годом выпуска автомобиля и мощностью были удалены выбросы.
Было обнаружено что датасет выбран из автомобилей Германии и на основании этого заменен на область в которой продавался автомобиль.
Выяснили что большая часть автомобилей представлена с пробегом 150000км, и из-за ограничения это значение является максимальным, но наверняка есть автомобили пробег которых значительно больше.
Проверили признаки на мультиколлениарность.
Добавили новый признак с разницей между пробегом автомобиля и медианным пробегом для такой же марки того же года. Что дало небольшой прирост качества.

Для обучения модели нами выбраны две можели на основе градиентного бустинга CatBoostRegressor и LGBMRegressor. И одна попроще Ridge regression.

По скрости обучения и предсказания лидирует Ridge. Обе модели градиентного бустинга проигрывают в скорости зато выигрывают в качестве предсказания. 

По совокупности результатов качества  и скорости предсказания, рекомендуется для разработки приложения использовать модель на основе CatBoostRegressor с расчитанными выше параметрами.